# Getting Started with Kotlin Notion Client

> **Note:** These notebooks currently use v0.2.0. Due to a binary incompatibility between the IntelliJ
> Kotlin Notebook kernel's bundled kotlinx-serialization and the version used by Ktor 3.4.0,
> v0.3.0 cannot be loaded in notebooks. This is a kernel-level limitation that will be resolved
> in a future kernel update. The library works correctly in all other environments.

This notebook introduces the Kotlin Notion Client library with basic operations:
- Setting up the client with authentication
- Retrieving your user information
- Retrieving a page by ID
- Understanding the Page object structure
- Basic error handling

## Prerequisites

Make sure you have set these environment variables:
- `NOTION_API_TOKEN` - Your Notion integration token
- `NOTION_TEST_PAGE_ID` - A test page ID you have access to

## Setup: Load Dependencies and Initialize Client

In [ ]:
// Load the Kotlin Notion Client library from Maven Central
@file:DependsOn("it.saabel:kotlin-notion-client:0.2.0")

// Import necessary classes
import it.saabel.kotlinnotionclient.NotionClient
import it.saabel.kotlinnotionclient.exceptions.NotionException
import kotlinx.coroutines.runBlocking

// Initialize the client with your API token
val apiToken = System.getenv("NOTION_API_TOKEN")
    ?: error("❌ NOTION_API_TOKEN environment variable not set")

val notion = NotionClient(apiToken)

println("✅ NotionClient initialized successfully!")
println("   Token: ${apiToken.take(10)}...")

## Example 1: Get Current User Information

Let's verify authentication works by retrieving information about your integration.

In [2]:
val user = runBlocking {
    notion.users.getCurrentUser()
}

println("✅ User information retrieved:")
println("   ID: ${user.id}")
println("   Name: ${user.name}")
println("   Type: ${user.type}")
println("   Avatar: ${user.avatarUrl ?: "No avatar"}")

✅ User information retrieved:
   ID: 261d43a1-b7fb-4a4a-8e44-0be9bdfc1186
   Name: kotlin-client-dev
   Type: BOT
   Avatar: No avatar


## Example 2: Retrieve a Page by ID

Now let's retrieve a specific page from your Notion workspace.

In [3]:
val pageId = System.getenv("NOTION_TEST_PAGE_ID")
    ?: error("❌ NOTION_TEST_PAGE_ID environment variable not set")

val page = runBlocking {
    notion.pages.retrieve(pageId)
}

println("✅ Page retrieved successfully!")
println("   ID: ${page.id}")
println("   Created: ${page.createdTime}")
println("   Last edited: ${page.lastEditedTime}")
println("   Archived: ${page.archived}")
println("   URL: ${page.url}")

✅ Page retrieved successfully!
   ID: 22dc63fd-82ed-80ee-a85a-ca78bc951fb1
   Created: 2025-07-11T08:10:00.000Z
   Last edited: 2025-10-16T11:01:00.000Z
   Archived: false
   URL: https://www.notion.so/Container-page-for-Kotlin-Notion-Client-integration-tests-22dc63fd82ed80eea85aca78bc951fb1


## Example 3: Exploring Page Properties

Pages in Notion can have various properties. Let's explore what properties this page has.

In [4]:
import it.saabel.kotlinnotionclient.models.pages.PageProperty

println("📋 Page Properties:")
page.properties.forEach { (name, property) ->
    println("   $name: ${property.type}")
    
    // Show some examples of accessing specific property types
    when (property) {
        is PageProperty.Title -> println("      → Title: ${property.plainText}")
        is PageProperty.RichTextProperty -> println("      → Text: ${property.plainText}")
        is PageProperty.Number -> println("      → Number: ${property.number}")
        is PageProperty.Checkbox -> println("      → Checked: ${property.checkbox}")
        is PageProperty.Select -> println("      → Selected: ${property.select?.name}")
        is PageProperty.Date -> println("      → Date: ${property.date?.start}")
        is PageProperty.Url -> println("      → URL: ${property.url}")
        else -> {} // Other types exist too
    }
}

📋 Page Properties:
   title: title
      → Title: Container page for Kotlin-Notion-Client integration tests


## Example 4: Understanding Page Parent

Every page has a parent - either another page, a data source, or a workspace.

In [5]:
import it.saabel.kotlinnotionclient.models.base.Parent

println("🔗 Page Parent Information:")

// Simple universal access - works for any parent type
println("   Parent ID: ${page.parent.id ?: "N/A (workspace)"}")

// Type-safe when you need specific parent details
when (val parent = page.parent) {
    is Parent.PageParent -> {
        println("   Type: Page")
        println("   Page ID: ${parent.pageId}")
    }
    is Parent.DataSourceParent -> {
        println("   Type: Data Source")
        println("   Data Source ID: ${parent.dataSourceId}")
    }
    is Parent.DatabaseParent -> {
        println("   Type: Database")
        println("   Database ID: ${parent.databaseId}")
    }
    is Parent.BlockParent -> {
        println("   Type: Block")
        println("   Block ID: ${parent.blockId}")
    }
    is Parent.WorkspaceParent -> {
        println("   Type: Workspace (root level)")
    }
}

🔗 Page Parent Information:
   Parent ID: 04645a33-97bc-4d6e-beba-a3442f58707a
   Type: Page
   Page ID: 04645a33-97bc-4d6e-beba-a3442f58707a


## Example 5: Error Handling

Let's demonstrate proper error handling when things go wrong.

In [6]:
// Try to retrieve a non-existent page
val fakePageId = "00000000-0000-0000-0000-000000000000"

try {
    runBlocking {
        notion.pages.retrieve(fakePageId)
    }
    println("This shouldn't happen!")
} catch (e: NotionException.ApiError) {
    if (e.status == 404) {
        println("✅ Caught expected error: Page not found")
        println("   Message: ${e.message}")
        println("   Status: ${e.status}")
        println("   Code: ${e.code}")
    } else {
        println("❌ Unexpected API Error: ${e.message}")
        throw e
    }
} catch (e: NotionException.AuthenticationError) {
    println("❌ Authentication failed: ${e.message}")
    println("   Check your NOTION_API_TOKEN is valid")
    throw e
} catch (e: NotionException) {
    println("❌ Unexpected error: ${e.message}")
    throw e
}

✅ Caught expected error: Page not found
   Message: API error: 404 (HTTP 404) - HTTP 404: Not Found. Response: {"object":"error","status":404,"code":"object_not_found","message":"Could not find page with ID: 00000000-0000-0000-0000-000000000000. Make sure the relevant pages and databases are shared with your integration.","request_id":"c33bcdbd-6d29-4799-b792-32ab9a17ecbc"}
   Status: 404
   Code: 404


## Example 6: Retrieve Page with Icon and Cover

Pages can have icons and cover images. Let's check if our page has any.

In [7]:
import it.saabel.kotlinnotionclient.models.pages.PageIcon
import it.saabel.kotlinnotionclient.models.pages.PageCover

println("🎨 Page Icon:")
when (val icon = page.icon) {
    is PageIcon.Emoji -> println("   Emoji: ${icon.emoji}")
    is PageIcon.CustomEmoji -> println("   Custom emoji: ${icon.customEmoji.name} (${icon.customEmoji.id})")
    is PageIcon.External -> println("   External URL: ${icon.external.url}")
    is PageIcon.File -> println("   File URL: ${icon.file.url} (expires: ${icon.file.expiryTime})")
    is PageIcon.FileUpload -> println("   Upload ID: ${icon.fileUpload.id}")
    null -> println("   No icon set")
}

println("\n🖼️  Page Cover:")
when (val cover = page.cover) {
    is PageCover.External -> println("   External URL: ${cover.external.url}")
    is PageCover.File -> println("   File URL: ${cover.file.url} (expires: ${cover.file.expiryTime})")
    is PageCover.FileUpload -> println("   Upload ID: ${cover.fileUpload.id}")
    null -> println("   No cover image set")
}

🎨 Page Icon:
   Emoji: 🧪

🖼️  Page Cover:
   No cover image set


## Cleanup

It's good practice to close the client when you're done.

In [8]:
notion.close()
println("🔒 NotionClient closed successfully")

🔒 NotionClient closed successfully


## Next Steps

Now that you understand the basics, explore:
- **[02-reading-databases.ipynb](./02-reading-databases.ipynb)** - Query databases and data sources
- **[03-creating-pages.ipynb](./03-creating-pages.ipynb)** - Create pages with properties
- **[04-working-with-blocks.ipynb](./04-working-with-blocks.ipynb)** - Build page content with blocks
- **[05-rich-text-dsl.ipynb](./05-rich-text-dsl.ipynb)** - Format text with the Rich Text DSL

## Tips for Exploration

- Modify the code cells and re-run them (`Shift + Enter`)
- Try retrieving different pages from your workspace
- Experiment with error handling for different scenarios
- Check the [documentation](../docs/README.md) for more details